# Load Data

In [1]:
import matplotlib.pyplot as plt
import xarray as xr

plt.rcParams['figure.figsize'] = [16, 3]

import sys
sys.path.append('/mnt/lustre/IAM851/jm1667/psc/python')
import psc

import logging
# logging.basicConfig(level=logging.DEBUG)

%config InlineBackend.figure_format = 'retina'
engine = "pscadios2"
species_names = ['e', 'i']

In [2]:
case = "case1"
B = 1
res = 512
mods = ["", "_v-1", '_v0', '_dup', '_T0', '_v2'][0]

path = f"/mnt/lustre/IAM851/jm1667/psc-runs/{case}/trials/B{B}_n{res}{mods}/"
length = {1:(1., .03, .03), 10:(1., .02, .02), .1:(1., .18, .18)}[B]
wholeSlice = slice(-length[1]/2, length[1]/2)
centerSlice = {10:slice(-.001, .001), 1:slice(-.003, .003), 0.1:slice(-.03, .03)}[B]

In [3]:
nframes = 100

# determine number of steps in file (will fail if run ran out of time)
with open(path + "params_record.txt") as records:
    fields_every = 200
    moments_every = 200
    for line in records:
        if line.startswith("nmax"):
            nsteps = int(line.split()[1])
        elif line.startswith("fields_every"):
            fields_every = int(line.split()[1])
        elif line.startswith("moments_every"):
            moments_every = int(line.split()[1])

stepsPerFrame = nsteps // nframes

print(f"nsteps in sim: {nsteps}")
print(f"nframes in animation = {nframes}")
print(f"steps per frame: {stepsPerFrame}")
print(f"directory to save in: {case}/B{B}_n{res}{mods}_{nframes}x{stepsPerFrame}")

nsteps in sim: 1000000
nframes in animation = 100
steps per frame: 10000
directory to save in: case1/B1_n512_100x10000


In [4]:
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

def openFile(fileName, step):
    return xr.open_dataset(path + f"{fileName}.{str(step).rjust(9,'0')}.bp", length=length, engine=engine, species_names=species_names)

def prepData(data):
    return data[0,:,:].transpose()

class DataLoader:
    def __init__(self):
        self.rGrid = None
        self.horGrid = None
        self.verGrid = None
    def getDataAndTime(self, frameIdx):
        if frameIdx == 0 and param.skipFirst:
            file = openFile(param.fileName, {'pfd':fields_every, 'pfd_moments':moments_every}[param.fileName])
        else:
            file = openFile(param.fileName, frameIdx*stepsPerFrame)

        if isinstance(param.varName, list):
            if param.combine == "magnitude":
                rawData = sum(prepData(file.__getattr__(var)) ** 2 for var in param.varName) ** .5
            elif param.combine == "sum":
                rawData = sum(prepData(file.__getattr__(var)) for var in param.varName)
            else:
                rawHor = prepData(file.__getattr__(param.varName[0]))
                rawVer = prepData(file.__getattr__(param.varName[1]))
                if self.rGrid is None:
                    self.rGrid = (rawHor.y **2 + rawHor.z**2)**.5
                    self.horGrid = rawHor.y
                    self.verGrid = rawHor.z
                if param.combine == 'radial':
                    rawData = (rawHor * self.horGrid + rawVer * self.verGrid) / self.rGrid
                elif param.combine == 'azimuthal':
                    rawData = (-rawHor * self.verGrid + rawVer * self.horGrid) / self.rGrid
                rawData = rawData.fillna(0)
        else:
            rawData = prepData(file.__getattr__(param.varName))
        return param.coef * rawData, file._attrs['time']
    def loadData(self):
        return [list(x) for x in zip(*[self.getDataAndTime(idx) for idx in range(nframes)])]

def setTitle(ax, param, time):
    ax.set_title(viewAdj + param.title + " (t={:.3f})".format(time))

class ParamMetadata:
    def __init__(self, title, vmin, vmax, colors, fileName, varName, coef=1, skipFirst=False, combine="magnitude"):
        self.title = title
        self.vmin = vmin
        self.vmax = vmax
        self.colors = colors
        self.fileName = fileName
        self.varName = varName
        self.coef = coef
        self.skipFirst = skipFirst
        self.combine = combine

In [5]:
paramId = 0
param = [
        ParamMetadata('Electron Density', 0, None, 'inferno', 'pfd_moments', 'rho_e', coef=-1),
        ParamMetadata('Ion Density', 0, None, 'inferno', 'pfd_moments', 'rho_i'),
        ParamMetadata('Y-ial E', -.0005, .0005, 'RdBu', 'pfd', 'ey_ec'),
        ParamMetadata('Y-ial B', -3e-8, 3e-8, 'RdBu', 'pfd', 'hy_fc', skipFirst=True),
        ParamMetadata('X-ial B', -B*1.2, B*1.2, 'RdBu', 'pfd', 'hx_fc'),
        ParamMetadata('Y-ial J', -.0005, .0005, 'RdBu', 'pfd', 'jy_ec', skipFirst=True),
        ParamMetadata('Radial J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='radial'),
        ParamMetadata('Azimuthal J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='azimuthal'),
        ParamMetadata('Electron Temperature', 0, None, 'inferno', 'pfd_moments', ['tyy_e', 'tzz_e'], skipFirst=False, combine='sum'),
        ][paramId]
print(f"quantity: {param.title}")

quantity: Electron Density


In [6]:
# load data
datas, times = DataLoader().loadData()

In [7]:
# slice data
activeSlice = [wholeSlice, centerSlice][0]

slicedDatas = [data.sel(y=activeSlice, z=activeSlice) for data in datas]
viewAdj = 'Central ' if activeSlice==centerSlice else ''
print(f"view: {viewAdj}= {activeSlice}")

vmax = param.vmax if not param.vmax is None else max(np.nanquantile(data.values, 1) for data in slicedDatas)
vmin = param.vmin if not param.vmin is None else min(np.nanquantile(data.values, 0) for data in slicedDatas)
if param.vmax is None and param.vmin is None:
    vmax = max(vmax, -vmin)
    vmin = -vmax

rGrid = (slicedDatas[0].y **2 + slicedDatas[0].z**2)**.5

view: = slice(-0.015, 0.015, None)


In [8]:
# view t
tidx = 20

%matplotlib ipympl
fig, ax = plt.subplots()

im = ax.imshow(slicedDatas[tidx], cmap=param.colors, vmin=vmin, vmax=vmax, origin='lower', extent=(activeSlice.start, activeSlice.stop, activeSlice.start, activeSlice.stop))
ax.set_xlabel('y')
ax.set_ylabel('z')
setTitle(ax, param, times[tidx])
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
fig.colorbar(im, ax=ax);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Stability

In [9]:
# fuzz: once a lmin is found, explore a radius of <fuzz> for potentially lower points, and keep going if any are found
def get_bounding_lmins_idxs(data, idx, fuzz=1):
    ilower = idx+1 # if idx is an lmin, it will count as the lower lmin
    i = 1
    foundDownhill = False
    while i <= fuzz:
        if ilower < i:
            ilower = ilower if foundDownhill else None
            break
        if ilower == len(data) or data[ilower - i] < data[ilower]:
            foundDownhill = True
            ilower -= i
            i = 1
        elif foundDownhill:
            i += 1
        else:
            ilower -= 1
        
    iupper = idx
    i = 1
    foundDownhill = False
    while i <= fuzz:
        if iupper + i >= len(data):
            iupper = iupper if foundDownhill else None
            break
        if data[iupper + i] < data[iupper]:
            foundDownhill = True
            iupper += i
            i = 1
        elif foundDownhill:
            i += 1
        else:
            iupper += 1

    return ilower, iupper

def get_lmin_idxs(data, fuzz=1):
    lmis = [0]
    while not lmis[-1] is None and lmis[-1] < len(data):
        lmis.append(get_bounding_lmins_idxs(data, lmis[-1], fuzz)[1])
    return [lm for lm in lmis if not lm is None]

In [10]:
%matplotlib ipympl
def norm(x):
    return xr.apply_ufunc(np.linalg.norm, x, input_core_dims=[['y', 'z']])

normsOfDiffs = [norm(data - slicedDatas[0]) for data in slicedDatas]
lmis = get_lmin_idxs(normsOfDiffs, 2)

plt.xlabel("Time")
plt.ylabel("2-Norm of Difference")
plt.title("Stability of " + viewAdj + param.title)

plt.plot(times, normsOfDiffs)
plt.plot([times[i] for i in lmis], [normsOfDiffs[i] for i in lmis], 'go')
plt.show()
print(lmis)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0, 15, 29, 43, 58, 72, 86, 99]


# Radial Dependence

In [11]:
import matplotlib

maxR = max(activeSlice.start, activeSlice.stop) * 2**.5
rStep = length[1] / 100

def getMeanAndStd(data, r):
    rslice = data.where((r <= rGrid) & (rGrid < r + rStep))
    return rslice.mean().item(), rslice.std().item()
rs = np.arange(0, maxR, rStep)

def getMeansAndStds(data):
    return tuple(zip(*[getMeanAndStd(data, r) for r in rs]))

def subtract(list1, list2):
    return [a1 - a2 for a1, a2 in zip(list1, list2)]

In [12]:
minmeanses, minstdses = tuple(zip(*[getMeansAndStds(slicedDatas[lmi]) for lmi in lmis]))
maxmeanses, maxstdses = tuple(zip(*[getMeansAndStds(slicedDatas[(lmi1 + lmi2) // 2]) for lmi1, lmi2 in zip(lmis[:-1], lmis[1:])]))
midmeanses, midstdses = tuple(zip(*[getMeansAndStds(slicedDatas[(3 * lmi1 + lmi2) // 4]) for lmi1, lmi2 in zip(lmis[:-1], lmis[1:])]))
means0, stds0 = getMeansAndStds(slicedDatas[0])

In [35]:
# mean values over a specific number of periods
lmii = 3
npers = 2
permeanses, perstdses = tuple(zip(*[getMeansAndStds(slicedDatas[lmi]) for lmi in range(lmis[lmii], lmis[lmii+npers])]))

In [36]:
# time average of mean values from a period
timeav = np.array(permeanses).mean(axis=0)

In [37]:
%matplotlib ipympl
from matplotlib import cm

plt.xlabel("Distance from Axis")
plt.ylabel(param.title)
# plt.title("Mean " + param.title + " vs Radius (t={:.2f})".format(times[dataIdx]))
plt.title("Mean " + param.title + " vs Radius")

##### plot error bars for a single time
# plt.errorbar(rs, means0, yerr=stds0, errorevery=(1,2), color="blue", ecolor="lightskyblue", elinewidth=1, capsize=1.5, label="t={:.2f}".format(0))
# plt.errorbar(rs, means, yerr=stds, errorevery=(0, 2), color="red", ecolor="lightcoral", elinewidth=1, capsize=1.5, label="t={:.2f}".format(times[dataIdx]))

##### plot sequence of lmins or whatever
# plt.plot(rs, means0, color="black", label="t={:.2f}".format(0))
# for i in range(len(minmeanses)):
#     plt.plot(rs, minmeanses[i], color=cm.Wistia(1-i/len(minmeanses)), label="t={:.2f}".format(times[lmis[i]]))
# for i in range(len(maxmeanses)):
#     plt.plot(rs, maxmeanses[i], color=cm.Wistia(1-i/len(maxmeanses)), label="t={:.2f}".format(times[(lmis[i] + lmis[i+1])//2]))
# for i in range(len(midmeanses)):
#     plt.plot(rs, midmeanses[i], color=cm.Wistia(1-i/len(midmeanses)), label="t={:.2f}".format(times[(3 * lmis[i] + lmis[i+1])//4]))

##### plot time average over a period
plt.plot(rs, timeav, label="t:[{:.2f},{:.2f}]".format(times[lmis[lmii]], times[lmis[lmii+npers]]))

# plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
##### plot times over a period
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X, Y = np.meshgrid(rs, times[lmis[lmii]:lmis[lmii+npers]])
# Z = np.array(permeanses)
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)

Z = np.array([subtract(permean, timeav) for permean in permeanses])
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, vmin=-.15, vmax=.15, linewidth=0, antialiased=False)

ax.set_xlim(0, .003)
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …